<a href="https://colab.research.google.com/github/obscraft23/SlimeChunkFinderAddonBE/blob/master/mcberepair_wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a ipynb wrapper of [mcberepair](https://github.com/reedacartwright/mcberepair).<br>
All the functions are experimental. <br>
You can delete pending ticks records and/or remote players' data.

In [ ]:
#@title <p> &darr; please push the &#9654;&#65039; button to start.<p> { vertical-output: true }

from IPython.utils import io
import os
import subprocess
import tqdm.notebook
from google.colab import files
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import datetime

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt}'
print('\033[1m'+"[Initial setting]"+'\033[0m'+" Please wait...")
# initial settings
try:
  print()
  with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      os.chdir("/content")
      os.system("git clone --recurse-submodules https://github.com/reedacartwright/mcberepair.git")
      os.chdir("mcberepair")
      pbar.update(40)
      os.system("cmake -S . -B build -DCMAKE_BUILD_TYPE=Release")
      os.system("cmake --build build --parallel")
      pbar.update(40)
      os.chdir("/content")
      os.system("mkdir test")
      os.chdir("test")
      os.system("ln -sf /content/mcberepair/build/mcberepair .")
      pbar.update(20)
  print("Initial setting done.")

except subprocess.CalledProcessError:
  print(captured)
  raise

#main
fname = "tmp.zip"
print()
print()
print('\033[1m'+"---------- [Functions] ----------"+'\033[0m')

def timestampJST(forLog=False,isJP=False):
  if isJP:
    t_delta = datetime.timedelta(hours=9)
    tz = datetime.timezone(t_delta, 'JST')
    tzname = "JST"
  else:
    t_delta = datetime.timedelta(hours=0)
    tz = datetime.timezone(t_delta, 'UTC')
    tzname = "UTC"
  
  dt = datetime.datetime.now(tz)
  if forLog:
    return str(dt.year)+"-"+str(dt.month).zfill(2)+"-"+str(dt.day).zfill(2)+" "+str(dt.hour).zfill(2)+":"+str(dt.minute).zfill(2)+":"+str(dt.second).zfill(2)+tzname
  else:
    timestamp = str(dt.year)+str(dt.month).zfill(2)+str(dt.day).zfill(2)+str(dt.hour).zfill(2)+str(dt.minute).zfill(2)+str(dt.second).zfill(2)+tzname
    logtimestamp = str(dt.year)+"-"+str(dt.month).zfill(2)+"-"+str(dt.day).zfill(2)+" "+str(dt.hour).zfill(2)+":"+str(dt.minute).zfill(2)+":"+str(dt.second).zfill(2)+tzname
    return timestamp,logtimestamp

def upload_zip(b):
  global fname
  os.system("rm -rf "+fname.replace(".zip","*"))
  uploaded = files.upload()
  try:
    with io.capture_output() as captured:
      
      fname = list(uploaded.keys())[0]
      os.system("rm -rf ./*.mcberepair.*UTC.mcworld")
      os.system("rm -rf ./*.mcberepair.*UTC.zip")
      os.system("rm -rf ./*.mcberepair.*JST.mcworld")
      os.system("rm -rf ./*.mcberepair.*JST.zip")
      os.system("rm -rf ./*.org")
      os.system("rm -rf "+fname.replace(".zip",""))
      os.system("unzip "+fname)
      os.system("cp -r "+fname.replace(".zip","")+" "+fname.replace(".zip",".org"))
      os.system("rm -rf "+fname)
    print(timestampJST(forLog=True)+" [Upload] "+'\033[32m'+os.path.basename(fname.replace(".zip",""))+'\033[0m'+" was uploaded and set as the target world.")

  except:
    pass

def delete_pticks(b):
  global fname
  if os.path.exists(fname.replace(".zip","")):
    try:
      with io.capture_output() as captured:
        os.system("./mcberepair listkeys "+fname.replace(".zip","")+" > list.tsv")
        os.system("awk '$6 == 51 && $2 > 4096 {print $1}' list.tsv > pending_ticks.tsv")
        os.system("./mcberepair rmkeys "+fname.replace(".zip","")+" < pending_ticks.tsv")
        nkeys = np.loadtxt("pending_ticks.tsv",delimiter=" ",dtype="|S").shape[0]
        os.system("rm -rf *.tsv")
      print(timestampJST(forLog=True)+" [Delete pticks] "+str(nkeys)+" keys were detected and deleted.")
    except subprocess.CalledProcessError:
      print(captured)
      raise
  else:
    print(timestampJST(forLog=True)+" "+"\033[33m"+"[Warn]"+"\033[0m"+" world data (.zip) seems NOT to be uploaded.")

def delete_remote_players(b):
  global fname
  if os.path.exists(fname.replace(".zip","")):
    try:
      with io.capture_output() as captured:
        os.system("./mcberepair listkeys "+fname.replace(".zip","")+" > list.tsv")
        os.system("awk '$0 ~ /player_/ {print $1}' list.tsv > remote_players.tsv")
        os.system("./mcberepair rmkeys "+fname.replace(".zip","")+" < remote_players.tsv")
        nkeys = np.loadtxt("remote_players.tsv",delimiter=" ",dtype="|S").shape[0]
        os.system("rm -rf *.tsv")
      print(timestampJST(forLog=True)+" [Delete remote players] "+str(nkeys)+" keys were detected and deleted.")

    except subprocess.CalledProcessError:
      print(captured)
      raise
  else:
    print(timestampJST(forLog=True)+" "+"\033[33m"+"[Warn]"+"\033[0m"+" world data (.zip) seems NOT to be uploaded.")

def reset_proc(b):
  global fname
  if os.path.exists(fname.replace(".zip","")):
    try:
      with io.capture_output() as captured:
        os.system("rm -rf "+fname.replace(".zip",""))
        os.system("cp -r "+fname.replace(".zip",".org")+" "+fname.replace(".zip",""))
      print(timestampJST(forLog=True)+" [Reset] all procedures were canceled.")

    except subprocess.CalledProcessError:
      print(captured)
      raise
  else:
    print(timestampJST(forLog=True)+" "+"\033[33m"+"[Warn]"+"\033[0m"+" world data (.zip) seems NOT to be uploaded.")

def download_zip(b):
  global fname
  if os.path.exists(fname.replace(".zip","")):
    try:
      with io.capture_output() as captured:
        
        os.system("rm -rf ./*.mcberepair.*UTC.zip")
        os.system("rm -rf ./*.mcberepair.*JST.zip")
        timestamp,logtimestamp = timestampJST()
        os.chdir(fname.replace(".zip",""))
        outputfname = os.path.basename( fname.replace(".zip",".mcberepair."+timestamp+".zip") )
        os.system("zip "+outputfname+" *")
        os.system("zip -r "+outputfname+" *")
        os.system("mv "+outputfname+" ../")
        os.chdir("../")

      files.download(outputfname)
      print(logtimestamp+" [Download] "+outputfname+" was downloaded.")

    except subprocess.CalledProcessError:
      print(captured)
      raise

  else:
    print(timestampJST(forLog=True)+" "+"\033[33m"+"[Warn]"+"\033[0m"+" world data (.zip) seems NOT to be uploaded.")

def download_mcworld(b):
  global fname
  if os.path.exists(fname.replace(".zip","")):
    try:
      with io.capture_output() as captured:
        
        os.system("rm -rf ./*.mcberepair.*UTC.mcworld")
        os.system("rm -rf ./*.mcberepair.*JST.mcworld")
        timestamp,logtimestamp = timestampJST()
        os.chdir(fname.replace(".zip",""))
        outputfname = os.path.basename( fname.replace(".zip",".mcberepair."+timestamp+".zip") )
        os.system("zip "+outputfname+" *")
        os.system("zip -r "+outputfname+" *")
        os.system("mv "+outputfname+" ../")
        os.chdir("../")
        os.system("mv "+outputfname+" "+outputfname.replace(".zip",".mcworld"))

      files.download(outputfname.replace(".zip",".mcworld"))
      print(logtimestamp+" [Download] "+outputfname.replace(".zip",".mcworld")+" was downloaded.")

    except subprocess.CalledProcessError:
      print(captured)
      raise

  else:
    print(timestampJST(forLog=True)+" "+"\033[33m"+"[Warn]"+"\033[0m"+" world data (.zip) seems NOT to be uploaded.")
  

button0 = widgets.Button(description="Delete pticks")
button1 = widgets.Button(description="Delete remote players")
button2 = widgets.Button(description="Reset")
button3 = widgets.Button(description="Download as zip")
button4 = widgets.Button(description="Download as mcworld")
button5 = widgets.Button(description="Upload zip file")
button0.on_click(delete_pticks)
button1.on_click(delete_remote_players)
button2.on_click(reset_proc)
button3.on_click(download_zip)
button4.on_click(download_mcworld)
button5.on_click(upload_zip)
display(button5)
print("  upload world data "+'\033[1m'+"(only one .zip file is aceptable.)"+'\033[0m')
print("   *compress your world folder (somthing like `TbLUY1uSXQA=') to .zip by yourself before uploading.")
print("   *"+'\033[1m'+"the file selector will appear at the bottom (below [Log]) of this window."+'\033[0m')
print()
display(button0)
print("  delete all pending ticks records above 4096 bytes in size.")
display(button1)
print("  delete all remote players' data.")
display(button2)
print("  reset all the procedures and reload the originally uploaded world.")
print()
display(button3)
print("  donlowad modified world as a .zip file.")
display(button4)
print("  donlowad modified world as a .mcworld file.")
print()
print('\033[1m'+"---------- [Log] ----------"+'\033[0m')